In [ ]:
from nnModel import getModel

model = getModel()
print(model.summary())

In [ ]:
import random

from trainingHelper import TrainingHelper
from trainingGame import loadTrainingGame
import nnHelper
import numpy as np

from os import listdir
from os.path import isfile, join

# filepath = "D:/GalconZero/Games/iteration_0"
filepath = "D:/GalconZero/Games"

BATCH_SIZE = 64

def getTrainingDataForNumGames(num):
    trainX = np.empty((0, 22, 14))
    trainY = [np.empty((0, 925)), np.empty((0,))]

    for _ in [1]:
        trainingGame = loadTrainingGame(filepath + "/game_1562210791.2397857_3.pickle")
        helper = TrainingHelper(trainingGame)    
        trainX = np.concatenate((trainX, helper.getTrainX()))
        [policy, value] = helper.getTrainY()
        trainY = [np.concatenate((trainY[0], policy)), np.concatenate((trainY[1], value))]

    print("Finished loading {} training files".format(num))
    return trainX, trainY

trainX, trainY = getTrainingDataForNumGames(1)

def getYAxisFlippedTrainingData(data):
    data = data.copy()
    for sample in data:
        for row in sample:
            row[7] *= -1
            row[10] *= -1
            row[13] *= -1
    return data

# this involves reversing ordering of each sample in trainX, then mapping indices somehow to fix trainY, both sends and redirects
def getXAxisFlippedTrainingData(data):
    raise NotImplementedError


In [ ]:
from trainingGame import loadTrainingGame
from trainingHelper import TrainingHelper
from mapHelper import mapHelper

game = loadTrainingGame("testGame.pickle")
exampleState = game.states[0]
mapHelper.setItems(exampleState)
helper = TrainingHelper(game) 

trainX = helper.getTrainX()
trainY = helper.getTrainY()


In [ ]:
# print(trainX[240])
for i in range(5, 20):
    print(game.states[240][i].owner)

In [ ]:
import numpy as np
print(trainY)
a = model.predict(np.array([trainX[0]]))
a[1][0][0]